# Logistic Regression

The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).

Link for the dataset: https://www.kaggle.com/datasets/rouseguy/bankbalanced

Cleaning and modifying data

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [37]:
# uploading dataset to juputer notebook
df = pd.read_csv("bank.csv")

In [38]:
# let's read data
df.head()

# I checked from this link https://archive.ics.uci.edu/dataset/222/bank+marketing
# what some of columns mean (their definition)

# default --> has credit in default? (binary: "yes","no")
# duration --> last contact duration, in seconds (numeric)
# campaign --> number of contacts performed during this campaign 
#and for this client (numeric, includes last contact)
# pdays --> number of days that passed by after the client was last contacted 
#from a previous campaign (numeric, -1 means client was not previously contacted)
# previous --> number of contacts performed before this campaign and for this client (numeric)
# poutcome --> outcome of the previous marketing campaign (categorical: "unknown","other","failure","success"

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


## My clean data plan:
1. Check all NaN values --> delete if there are 
2. Check duplicates --> delete if there are 

3. Check the job column --> use clustering or other methods to modify it into numeric
4. marital column --> OneHotEncoder
5. education column --> OneHotEncoder
6. default column --> LabelEncoder
7. housing column --> LabelEncoder
8. loan column --> LabelEncoder
9. contact column --> OneHotEncoder 
10. month column --> use clustering or other methods to modify it into numeric
11. poutcome column --> OneHotEncoder 
12. deposit column --> LabelEncoder

13. Remove outliers

In [39]:
# 1. Check all NaN values --> delete if there are 
df.isna().sum()

# suprisingly, zero :)

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [40]:
# 2. Check duplicates --> delete if there are 
# let's check whether we have duplicates
# or one more surprise awaiting us :)
df.duplicated().sum()

# yes, life is full of surprises :)

0

In [41]:
# 3. Check the job column --> use clustering or other methods to modify it into numeric



In [42]:
# All columns that need the same method can be done in one step
# 4. marital column --> OneHotEncoder
# 5. education column --> OneHotEncoder
# 9. contact column --> OneHotEncoder 
# 11. poutcome column --> OneHotEncoder 

# this makes multiple columns with the variable (Separate for yes/no)
from sklearn.preprocessing import OneHotEncoder
variables = ['marital', 'education', 'contact', 'poutcome']

# use encoder
encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
one_hot_encoded = encoder.fit_transform(df[variables]).astype(int)
df = pd.concat([df,one_hot_encoded],axis=1).drop(columns=variables)

In [43]:
# quite many new columns we have now
# we can delete one column in every category
df.head()

,age,job,default,balance,housing,loan,day,month,duration,campaign,...,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,59,admin.,no,2343,yes,no,5,may,1042,1,...,1,0,0,0,0,1,0,0,0,1
1,56,admin.,no,45,no,no,5,may,1467,1,...,1,0,0,0,0,1,0,0,0,1
2,41,technician,no,1270,yes,no,5,may,1389,1,...,1,0,0,0,0,1,0,0,0,1
3,55,services,no,2476,yes,no,5,may,579,1,...,1,0,0,0,0,1,0,0,0,1
4,54,admin.,no,184,no,no,5,may,673,2,...,0,1,0,0,0,1,0,0,0,1


In [ ]:
# drop unnesecary columns
df = df.drop("education_unknown", "contact_unknown", "education_unknown",axis=1)

In [44]:
# 10. month column --> use clustering or other methods to modify it into numeric

In [45]:
# All columns that need the same method can be done in one step
# 6. default column --> LabelEncoder
# 7. housing column --> LabelEncoder
# 8. loan column --> LabelEncoder
# 12. deposit column --> LabelEncoder

# this just converts the value of column to 0 or 1
# factorize in pandas works too, but only one column at a time
from sklearn.preprocessing import LabelEncoder
variables = ['default', 'housing', 'loan', 'deposit']
encoder = LabelEncoder()
df[variables] = df[variables].apply(encoder.fit_transform)

In [46]:
df.head()

,age,job,default,balance,housing,loan,day,month,duration,campaign,...,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,59,admin.,0,2343,1,0,5,may,1042,1,...,1,0,0,0,0,1,0,0,0,1
1,56,admin.,0,45,0,0,5,may,1467,1,...,1,0,0,0,0,1,0,0,0,1
2,41,technician,0,1270,1,0,5,may,1389,1,...,1,0,0,0,0,1,0,0,0,1
3,55,services,0,2476,1,0,5,may,579,1,...,1,0,0,0,0,1,0,0,0,1
4,54,admin.,0,184,0,0,5,may,673,2,...,0,1,0,0,0,1,0,0,0,1


/var/folders/lj/tcwl5glj0blfftjn4df0129r0000gn/T/ipykernel_84447/1915093933.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("education_unknown", "education_unknown", "education_unknown",axis=1)


TypeError: DataFrame.drop() got multiple values for argument 'axis'